<img src="../../shared/img/banner.svg"></img>

# Lab 07 - ANOVA: Distribution, Robustness, and Power

In [ ]:
%matplotlib inline

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns

import utils.util as util

## Learning Objectives

1. Understand what the null model of an ANOVA is and what the sampling distribution of the $F$-statistic looks like.
1. Know the core assumptions of ANOVA and how violating them effects the false positive rate.
1. Understand how to use sampling to perform a power analysis on a proposed one-way ANOVA.

## Introduction

In this lab, we'll get familiar with the data model underlying the ANOVA statistical test and see what happens when that model is wrong.

As our running example, we'll consider an experiment looking at the effect of caffeine on alertness.

The key function for this is the `util.specify_caffeine_ground_truth` function, from this lab's `utils`.
It defines a `Model` for our data, determined by its parameters.
Each time a sample is drawn from this `Model`, the result is the entire dataset from an experiment.

## Section 1 - The Null Model and False Positives

First, let's see what the data looks like when there's no effect. This is known as the _null model_ and it's the model corresponding to the _null hypothesis_.

In the cell below, define the `no_effect_model` by setting the `effect_size_caffeine` appropriately.
What is the effect size for the null model?

Next, generate a sample from the model and plot the data. Note: `samples_to_df` will come in handy here.

Display the data in at least two ways: in one, use the `had_caffeine` variable to distinguish the categories visually (use colors, line or marker styles, put them in a different plot, whichever you prefer).
In another, use a similar visual style but don't separate out data points by category.

#### Q Describe what you see in the data in the cell below.
For example, you might answer the following: how is the data distributed? Is the data spread out or clustered? Does it have a familiar shape? Looking only at the plot that doesn't separate out the data points into categories, can you tell which alertness scores came from subjects who had caffeine and which ones didn't? Looking at the plot where the data _is_ separated by whether subjects had caffeine or not, do the two data distributions look the same or different? In answering these questions, it'll be helpful to run the "experiment" multiple times, comparing the visualizations across samples.

Null Hypothesis Significance Testing (NHST), the standard style of inferential thinking in psychology and other sciences, tests the chance that a given statistic (for ANOVA, $F$) would be at least as extreme as the statistic observed in the data under the null model.

Use `util.simulate_experiments_and_run_anovas` to generate data and run an ANOVA NHST for a single experiment (`num_experiments=1`).
This function returns a list of dataframes containing the experiment results as the first output and a list of dataframes containing the results of the ANOVA in the second argument.

View the results of your ANOVA (the $p$ value is in the column `PR(>F)`), then answer the question below.

#### Q Report the results of your test in the APA style.

It is likelier than not that you did not report a significant effect of caffeine on alertness.
In the cell below, use a `groupby` to calculate the means for subjects who had and did not have caffeine.

#### Q Compare the averages. Are they close together or far apart?

We can't say whether a given test will give a significant result, but we can say what will happen if we run the experiment many times.

In the cell below, run the experiment 100 times (`num_experiments=100`).
Then, collect the $p$ values and calculate the fraction that are less than $0.05$.

#### Q Say you ran the experiment 1000 times and then calculated this fraction. What would you expect it to be?

Next, in the cell below, find the smallest $p$ value in all of the experiments you simulated and calculate the means for each group in that experiment.

#### Q How does the difference between these means compare to the difference you found in your original experiment? Explain why you should expect this difference.

### What if the Model was Mis-Specified?

In order to be able to compute $p$ values, the ANOVA test makes several assumptions about the data.
These assumptions are used to determine the distribution of the $F$ statistic under the null model,
which is then used to compute $p$.
When the assumptions are violated, the $p$ values we calculate based on those assumptions can be incorrect.

For one, the ANOVA assumes that the data within each category comes from a Gaussian distribution.
One common way this is violated is if the data contains _outliers_, or some values that are very different from others.

We'll simulate this by swapping out the Gaussian distribution in our model for a [Student's $t$](https://en.wikipedia.org/wiki/Student%27s_t-distribution) distribution.
This can be done by setting the `outliers` argument to `True` in `util.specify_caffeine_ground_truth`.

As above, generate some data from this new model, and look at the data.

#### Q Compare and contrast this data to the data you observed above.

Now, in the cell below, simulate 100 experiments with outliers and calculate the fraction that have $p<0.05$.
This might take a minute.

#### Q Is the value you got still close to $0.05$ (close being within ±0.03)?

The ANOVA also assumes that the variance in each category is the same.
This is known as _homoscedasticity_: _homo_ for "same" and _scedastic_ for "spread".
The opposite is _heteroscedasticity_: _hetero_ for "different".

In the cell below, generate some data with different variances in the two groups. Set the value of the `heteroscedasticity` argument to `2` to make the variance of the group that received caffeine four times larger.
Again, visualize the data.

#### Q Compare and contrast this data to all the other data you've seen so far.

In the cell below, run 100 experiments with heteroscedasticity and again calculate the fraction that had $p$ value below $0.05$.

#### Q Is the value you got still close to $0.05$?

Previously, all of the experiments have used 50 subjects in each group: a _balanced design_.
This is common in scientific settings, where we assign individuals to groups, but uncommon elsewhere.

It's possible to run ANOVA on groups with different sizes, but the behavior of the test might be different.

In the cell below, repeat the analysis where you run 100 experiments and check which give a significant result, but add the argument `subject_split=0.25` to set the fraction of individuals given caffeine to $0.25$ (the default value is `0.5`).

#### Q Is the value you got still close to $0.05$?

A statistical test is said to be _robust_ if the computed $p$ values stay roughly correct even when the assumptions are violated.

#### Q Based on your results, would you consider ANOVA to be robust? Feel free to attach conditions, e.g. "robust to outliers but not heteroscedasticity".

## Section 2 - Power Analysis

The previous section considered the chance that, if the null hypothesis is true and there is no effect, our ANOVA test would give us the wrong answer.

The other probability of interest to us as statisticians is the chance that, if the null hypothesis _isn't_ true, our ANOVA test correctly tells us that there is an effect.
This probability is called the _power_ of the test.
We'd like to obtain a high power while minimizing the expense of the experiment.

There's one problem: if we don't know the effect size, we don't have a complete model,
so we can't draw samples.
This is reflected in the fact that `specify_ground_truth` requires a value for `effect_size_caffeine`.
And we don't know the effect size, usually, because otherwise we wouldn't be running an experiment.

In order to compute the power, then, we need to have a guess for what the size of the effect is.
That allows us to have a concrete, complete model for our data,
just as we had in the case of the null model, where the effect size was 0.
In practice, we can either
1) collect a small amount of "pilot" data ("pilot" because it guides the design of the larger experiment), using the effect size calculated from that smaller dataset to calculate power
2) try a bunch of reasonable values and see what range of powers we get.

Below, we'll try the second strategy.
The answer will turn out to be that power is fairly sensitively dependent on the effect size,
which means picking the optimal value for the number of subjects will be challenging.

Note that power calculation, in our simulation framework,
is exactly the same as calculating the false positive rate:
compute the fraction of $p$s that were below `0.05`.

In the cell below, define a model with `effect_size_caffeine=0.7` and `num_subjects=100` and compute the power.

In [ ]:
moderate_effect_model = util.specify_caffeine_ground_truth(effect_size_caffeine=0.7, num_subjects=100)

moderate_effect_data_dfs, moderate_effect_anova_tables = util.simulate_experiments_and_run_anovas(
    moderate_effect_model, num_experiments)

moderate_effect_ps = get_ps(moderate_effect_anova_tables)

np.mean(moderate_effect_ps < 0.05)

#### Q Will changing the number of subjects to `10` increase or decrease the power? Write your answer here (it's OK to be wrong!), then run a simulation and report and explain the results.

Note that power can also, for some tests, be determined by explicit calculation, instead of simulation.
Again, in this course we go for the "one-size-fits-all" approach of simulating everything,
rather than looking for case-by-case explicit methods that require lots of mathematical sophistication.

The advantage, again, is that we can more easily change the underlying model and recalculate the quantities of interest.
If you have time, try seeing the effect of heteroscedasticity and outliers on power.
This is much harder to calculate explicitly!

### Plotting the Sampling Distribution of $F$ and $p$

Traditional statistical tests like the ANOVA are built on knowledge of the explicit form of _sampling distributions_ of certain statistics.
For ANOVA, the statistic is the $F$ value, whose distribution has a particularly gnarly explicit form:
$$
p(F; d_1, d_2) = \frac{1}{B\left(\frac{d_1}{2}, \frac{d_2}{2}\right)}
    \left(\frac{d_1}{d_2}\right)^{\frac{d_1}{2}}
    F ^ {\frac{d_1}{2} - 1}
    \left(1 + \frac{d_1}{d_2}F\right)^{-\frac{d_1 + d_2}{2}}
$$
where $d_1$ and $d_2$ are the number of categories and the number of subjects, respectively,
and $B$ is the [beta function](https://en.wikipedia.org/wiki/Beta_function):
$$
B(x, y) = \int_0^1 t^{x-1}(1-t)^{y-1}dt
$$

This formula is presented only to impress upon the reader just how difficult the explicit, mathematical approach to statistics can be.
Defining $F$ and determining its distribution was a major contribution by Sir Ronald Fisher to the early practice of statistics.
For decades, these values had to be calculated, painstakingly, and printed in tables.

Rather than working through these derivations as Fisher had to laboriously do,
we can simulate our way to success.

Using `simulate_experiments_and_run_anova`,
estimate the sampling distributions for $F$ and $p$
for effect sizes `0` (no effect), `0.1` (small effect), `0.7` (moderate effect), and `1.5` (big effect).
That is, take the values of the `p` and `F` statistics from the `anova_tables` that come out of that function
and compute histograms for each statistic for each effect size from no effect to a big effect.

#### Q Compare the sampling distributions of $F$ for no effect and for effect size 0.1. How much do they overlap? What does this imply about the power of the ANOVA to detect the small effect?

#### Q Compare the sampling distributions of $p$ for no effect and for increasing effect sizes. What is different about them? How does this connect to the power of the ANOVA test?